In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import os
import sys
import pyproj
sys.path.append("../src/if_license_plates_could_talk")
import data

In [4]:
df_geo = gpd.read_file(os.path.join("..", "geo", "shapefiles", "VG250_KRS3", "VG250_KRS.shp"))
df = data.db.get_data()

epsg:4326
epsg:4326


In [70]:
df.head()

,kreis_key,kreis_name,income_pp_2000,income_pp_2001,income_pp_2002,income_pp_2003,income_pp_2004,income_pp_2005,income_pp_2006,income_pp_2007,...,population_2010,population_2011,population_2012,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019
0,09361,Amberg,16120.0,16451.0,16433.0,16878.0,17318.0,17724.0,18326.0,18841.0,...,43755.0,41912.0,41684.0,41585.0,41564.0,41698.0,42105.0,42298.0,42109.0,42089.0
1,09561,Ansbach,17507.0,18162.0,17736.0,17849.0,18038.0,18022.0,17955.0,18110.0,...,40374.0,39574.0,39642.0,39762.0,39925.0,40585.0,41346.0,41592.0,41750.0,41823.0
2,09661,Aschaffenburg,17228.0,17972.0,17975.0,18703.0,19305.0,19802.0,20393.0,20802.0,...,68706.0,67398.0,67576.0,67763.0,68006.0,68577.0,69087.0,69558.0,70228.0,70765.0
3,09761,Augsburg,15590.0,16010.0,15969.0,16279.0,16471.0,16432.0,16547.0,16754.0,...,263903.0,268376.0,271051.0,274621.0,278827.0,283743.0,287979.0,291218.0,293993.0,295859.0
4,08211,Baden-Baden,25133.0,25229.0,23987.0,23945.0,23923.0,25003.0,26262.0,27180.0,...,54456.0,52404.0,52505.0,52799.0,53177.0,53751.0,54234.0,54513.0,54921.0,55154.0


In [71]:
df.head()

,kreis_key,kreis_name,income_pp_2000,income_pp_2001,income_pp_2002,income_pp_2003,income_pp_2004,income_pp_2005,income_pp_2006,income_pp_2007,...,population_2010,population_2011,population_2012,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019
0,09361,Amberg,16120.0,16451.0,16433.0,16878.0,17318.0,17724.0,18326.0,18841.0,...,43755.0,41912.0,41684.0,41585.0,41564.0,41698.0,42105.0,42298.0,42109.0,42089.0
1,09561,Ansbach,17507.0,18162.0,17736.0,17849.0,18038.0,18022.0,17955.0,18110.0,...,40374.0,39574.0,39642.0,39762.0,39925.0,40585.0,41346.0,41592.0,41750.0,41823.0
2,09661,Aschaffenburg,17228.0,17972.0,17975.0,18703.0,19305.0,19802.0,20393.0,20802.0,...,68706.0,67398.0,67576.0,67763.0,68006.0,68577.0,69087.0,69558.0,70228.0,70765.0
3,09761,Augsburg,15590.0,16010.0,15969.0,16279.0,16471.0,16432.0,16547.0,16754.0,...,263903.0,268376.0,271051.0,274621.0,278827.0,283743.0,287979.0,291218.0,293993.0,295859.0
4,08211,Baden-Baden,25133.0,25229.0,23987.0,23945.0,23923.0,25003.0,26262.0,27180.0,...,54456.0,52404.0,52505.0,52799.0,53177.0,53751.0,54234.0,54513.0,54921.0,55154.0


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 472
Data columns (total 79 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kreis_key        473 non-null    object 
 1   kreis_name       400 non-null    object 
 2   income_pp_2000   401 non-null    float64
 3   income_pp_2001   401 non-null    float64
 4   income_pp_2002   401 non-null    float64
 5   income_pp_2003   401 non-null    float64
 6   income_pp_2004   401 non-null    float64
 7   income_pp_2005   401 non-null    float64
 8   income_pp_2006   401 non-null    float64
 9   income_pp_2007   401 non-null    float64
 10  income_pp_2008   401 non-null    float64
 11  income_pp_2009   401 non-null    float64
 12  income_pp_2010   401 non-null    float64
 13  income_pp_2011   401 non-null    float64
 14  income_pp_2012   401 non-null    float64
 15  income_pp_2013   401 non-null    float64
 16  income_pp_2014   401 non-null    float64
 17  income_pp_2015  

In [73]:
df_comb = df_geo.merge(df, left_on = "RS", right_on = "kreis_key", how = "left")

In [74]:
feature_info = {
            "crimes_pp": {
                "title": "Erfasste Straftaten",
                "label": "Straftaten / EW"
            },
            "income_pp": {
                "title": "Verfügbares Einkommen der priv. Haushalte",
                "label": "Euro / EW"
            },
            "population": {
                "title": "Bevölkerung",
                "label": "EW"
            }
}

In [75]:
def generate_static_map(feature, year):
    col = f"{feature}_{year}"
    title = feature_info[feature]["title"]
    fig = px.choropleth(df_comb, geojson=df_comb.geometry, locations=df_comb.index, color=col, scope="europe",
                            color_continuous_scale="gray_r",
                            title=f"{title} ({year})",
                            range_color=(df_comb[col].min(
                            )*0.8, df[col].max()),
                            labels={
                                col: feature_info[feature]["label"]
                            })
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(hoverlabel={"bgcolor": "white"}, title_font_color="black", font_color="black")

    fig.add_annotation(dict(font=dict(color='black',size=11),
                                        x=-0.1,
                                        y=-0.12,
                                        showarrow=False,
                                        text="Quelle: BKA, Regionaldatenbank Deutschland",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))

    fig.write_image(os.path.join("..", "output", f"{col}.png"), scale=1)
    fig.write_html(os.path.join("..", "output", f"{col}.html"))

In [76]:
generate_static_map("income_pp", 2018)
generate_static_map("crimes_pp", 2018)
generate_static_map("population", 2018)